# Structurer et explorer des données textuelles

Notebook Introduction au traitement du langage naturel - 15/05/2025 - Émilien Schultz

## Les données

Données questions au gouvernement https://www.data.gouv.fr/fr/datasets/questions-au-gouvernement/

Télécharger les données et les décompresser dans un répertoire `data/` dans l'espace de travail

## Les bibliothèques

- `pandas` pour la manipulation de données
- `nltk` pour le traitement de texte
- `matplotlib` pour la visualisation
- `scikit-learn` pour le traitement de texte et la modélisation


In [3]:
#!pip install pandas nltk scikit-learn matplotlib

## Structurer les données

Avoir des données facilement manipulables

Regarder ce qu'il y a dans un fichier

In [5]:
with open("./data/json/QANR5L15QG1001.json","r") as f:
    print(f.read())

{"question": {"@xmlns": "http://schemas.assemblee-nationale.fr/referentiel", "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance", "@xsi:type": "QuestionGouvernement_Type", "uid": "QANR5L15QG1001", "identifiant": {"numero": "1001", "regime": "5eme R\ufffdpublique", "legislature": "15"}, "type": "QG", "indexationAN": {"rubrique": "agriculture", "teteAnalyse": null, "ANALYSE": {"ANA": "interdiction du dim\u00e9thoate"}}, "auteur": {"identite": {"acteurRef": "PA721876", "mandatRef": "PM733719"}, "groupe": {"organeRef": "PO730964", "abrege": "LAREM", "developpe": "La R\u00e9publique en Marche"}}, "minInt": {"abrege": "Agriculture et alimentation", "developpe": "Minist\u00e8re de l'agriculture et de l'alimentation"}, "minAttribs": {"minAttrib": {"denomination": {"abrege": "Agriculture et alimentation", "developpe": "Minist\u00e8re de l'agriculture et de l'alimentation"}}}, "textesReponse": {"texteReponse": {"infoJO": {"typeJO": "JO_DEBAT", "dateJO": "14/06/2018"}, "texte": "</p><p alig

### Etape de préparation

Je veux :
- ouvrir chaque fichier
- récupérer le champ "text" de la question
- faire un tableau qui contient toutes les données textuelles pour ensuite faire l'analyse

In [6]:
import pandas as pd
import glob
import json

#### Faisons-le pour un seul fichier

In [9]:
fichiers = glob.glob("./data/json/*")
fichiers[0]

'./data/json/QANR5L15QG3063.json'

In [12]:
fichier = json.load(open(fichiers[100], "r"))
fichier["question"]["identifiant"]["legislature"]

'15'

On veut récupérer un tableau avec :

- la date de la question
- ~~la législature~~
- le texte de la question

In [13]:
numero = fichier["question"]["identifiant"]["numero"],
# legislature = fichier["question"]["identifiant"]["legislature"],
date = fichier["question"]["textesReponse"]["texteReponse"]["infoJO"]["dateJO"]
texte = fichier["question"]["textesReponse"]["texteReponse"]["texte"]

In [17]:
numero, date

(('4837',), '23/02/2022')

Faire une fonction qui le fait pour n'importe quel fichier

In [18]:
def get_info(fichier):
    """
    Fonction qui extrait les informations d'une question parlementaire
    """
    try:
        numero = fichier["question"]["identifiant"]["numero"]
        # legislature = fichier["question"]["identifiant"]["legislature"]
        date = fichier["question"]["textesReponse"]["texteReponse"]["infoJO"]["dateJO"]
        texte = fichier["question"]["textesReponse"]["texteReponse"]["texte"]
        return {
            "numero": numero,
            # "legislature": legislature,
            "date": date,
            "texte": texte
        }
    except Exception as e:
        print(f"Erreur lors de l'extraction des informations : {e}")
        return None

In [20]:
# get_info(json.load(open(fichiers[100], "r")))

Application au corpus

In [21]:
# j'applique la fonction sur chaque élément du corpus
t = [get_info(json.load(open(fichier, "r"))) for fichier in fichiers]
# je filtre les valeurs nulles
t = [i for i in t if i is not None]
# je mets sous une forme de dataframe
df = pd.DataFrame(t)

Erreur lors de l'extraction des informations : 'NoneType' object is not subscriptable


In [25]:
df.head()

,numero,date,texte
0,3063,03/06/2020,"</p><p align=""CENTER""> RÔLE DES COLLECTIVITÉS ..."
1,3599,09/12/2020,"</p><p align=""CENTER""> CONTRÔLE DES EXPORTATIO..."
2,89,10/08/2017,"</p><p align=""CENTER""> TAXE DE SÉJOUR <a name=..."
3,348,29/11/2017,"</p><p align=""CENTER""> LISTE NOIRE DES PARADIS..."
4,3433,21/10/2020,"</p><p align=""CENTER""> LUTTE CONTRE LE FINANCE..."


Une fois qu'on a les données, on peut s'inquiéter de leur qualité

-> on veut pas les balises HTML

## Nettoyer les données (préprocessing)

Enlever les balises HTML et les autres soucis : enlever tous les éléments qui ont la forme \<XXXXXX>

Pour cela on peut utiliser une regex. 

### Remarque sur les regex

- bibliothèque `re` ou `regex`
- `re.sub` pour remplacer une partie d'une chaîne par une autre
- `re.findall` pour trouver toutes les occurrences d'une regex dans une chaîne
- [Aller voir une cheatsheet 🤓](https://www.pythoncheatsheet.org/cheatsheet/regular-expressions) ou utiliser un [site dédié](https://regex101.com/)

In [28]:
import re
re.sub(r"<.*?>", "", "Ceci est un <b>test</b> <coucou> <cou")

'Ceci est un test  <cou'

Appliquer au corpus : on définit une fonction de nettoyage

In [44]:
def clean_text(text):
    """
    Fonction qui nettoie le texte en supprimant les balises HTML et les espaces inutiles
    :param text: le texte à nettoyer
    :return: le texte nettoyé
    """
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"\s\s+", " ", text)
    # text = text.capitalize()
    # autres règles
    return text.strip()

On teste la fonction

In [42]:
clean_text("Ceci est un texte.    <p>avec un paragraphe</p> COUCOU")

'Ceci est un texte. avec un paragraphe coucou'

Application au corpus

In [45]:
df["texte_net"] = df["texte"].apply(clean_text)
df[["texte","texte_net"]].head(5)

,texte,texte_net
0,"</p><p align=""CENTER""> RÔLE DES COLLECTIVITÉS ...",RÔLE DES COLLECTIVITÉS LOCALES DANS LA LUTTE C...
1,"</p><p align=""CENTER""> CONTRÔLE DES EXPORTATIO...",CONTRÔLE DES EXPORTATIONS D'ARMEMENT M. le pré...
2,"</p><p align=""CENTER""> TAXE DE SÉJOUR <a name=...",TAXE DE SÉJOUR M. le président. La parole est ...
3,"</p><p align=""CENTER""> LISTE NOIRE DES PARADIS...",LISTE NOIRE DES PARADIS FISCAUX M. le présiden...
4,"</p><p align=""CENTER""> LUTTE CONTRE LE FINANCE...",LUTTE CONTRE LE FINANCEMENT DU TERRORISME M. l...


In [40]:
#df.loc[1,"texte"]

In [47]:
df.to_csv("./data/dataframe.csv")

In [50]:
# ls data

## Analyse à l'échelle des mots

### Chercher la présence d'un mot

Les bases de la fouille de données. Quels sont les questions qui parlent d'intelligence artificielle ?

In [73]:
filtre = df["texte"].str.contains("coucou")
# df[filtre].loc[2912, "texte"]

Chercher un contexte d'un mot avec uen expression régulière

In [75]:
re.findall(".{40}coucou.{40}", df[filtre].loc[2912, "texte"])

["rante ans, <i>Vol au-dessus d'un nid de coucou</i> dénonçait une vision totalitaire de"]

In [78]:
(df["texte"].str.lower()
            .str.contains("intelligence artificielle")
            .sum())

35

Et si on cherche plusieurs termes ?

In [88]:
termes = ["intelligence artificielle", "IA", "algorithme"]

(df["texte"].str.lower()
            .str.contains("|".join(termes))
            .sum())

55

Faire une recherche sur toutes les variables possibles de l'IA

- intelligence artificelle
- algorithme
- AI
- ...

### Tokenisation

Découper un texte

#### Utiliser les regex

In [89]:
import re
word_pattern = r"\w+"
tokens = re.findall(word_pattern, "Ceci est un test")
tokens

['Ceci', 'est', 'un', 'test']

In [93]:
df["texte_net"].apply(lambda x: re.findall(r"\w+",x.lower()))

0       [rôle, des, collectivités, locales, dans, la, ...
1       [contrôle, des, exportations, d, armement, m, ...
2       [taxe, de, séjour, m, le, président, la, parol...
3       [liste, noire, des, paradis, fiscaux, m, le, p...
4       [lutte, contre, le, financement, du, terrorism...
                              ...                        
4845    [fermetures, de, classes, en, milieu, rural, m...
4846    [stratégie, en, matière, de, dépistage, m, le,...
4847    [situation, de, l, entreprise, earta, m, le, p...
4848    [limitation, de, la, vitesse, autorisée, sur, ...
4849    [application, de, la, loi, denormandie, m, le,...
Name: texte_net, Length: 4850, dtype: object

#### Utiliser une première bibliothèque : `nltk`

In [94]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

word_tokenize("Ceci est un test")

[nltk_data] Downloading package punkt to /Users/emilien/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['Ceci', 'est', 'un', 'test']

In [95]:
df["texte_net"].apply(word_tokenize)

0       [RÔLE, DES, COLLECTIVITÉS, LOCALES, DANS, LA, ...
1       [CONTRÔLE, DES, EXPORTATIONS, D'ARMEMENT, M., ...
2       [TAXE, DE, SÉJOUR, M., le, président, ., La, p...
3       [LISTE, NOIRE, DES, PARADIS, FISCAUX, M., le, ...
4       [LUTTE, CONTRE, LE, FINANCEMENT, DU, TERRORISM...
                              ...                        
4845    [FERMETURES, DE, CLASSES, EN, MILIEU, RURAL, M...
4846    [STRATÉGIE, EN, MATIÈRE, DE, DÉPISTAGE, M., le...
4847    [SITUATION, DE, L'ENTREPRISE, EARTA, M., le, p...
4848    [LIMITATION, DE, LA, VITESSE, AUTORISÉE, SUR, ...
4849    [APPLICATION, DE, LA, LOI, DENORMANDIE, M., le...
Name: texte_net, Length: 4850, dtype: object

### Quels sont les termes les plus fréquents ?

In [97]:
from collections import Counter

In [99]:
compteur = Counter([j for i in list(df["texte_net"].apply(word_tokenize)) for j in i])

In [100]:
compteur.most_common(20)

[(',', 242083),
 ('.', 191144),
 ('de', 187067),
 ('la', 107423),
 ('le', 89111),
 ('et', 87945),
 ('les', 81402),
 ('à', 76522),
 ('des', 76003),
 ('du', 48548),
 ('que', 43008),
 ('en', 41947),
 ('sur', 39774),
 ('pour', 33251),
 ('est', 31121),
 ('qui', 30337),
 ('ministre', 30064),
 ('nous', 28026),
 ('!', 25437),
 ('dans', 24672)]

### Quelles sont les expressions qui reviennent le plus souvent ?

Utilisons les bigrammes et les trigrammes

In [102]:
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

def generate_bigrams_nltk(text):
    tokens = word_tokenize(text.lower())
    bigrams = list(ngrams(tokens, 2))
    return bigrams

#generate_bigrams_nltk(df["texte_net"].iloc[0])

In [112]:
word_tokenize("ici-même, y'a-t'il")

['ici-même', ',', 'y', "'", "a-t'il"]

#### Enlever les stop words

In [107]:
nltk.download("stopwords")

from nltk.corpus import stopwords

french_stopwords = list(set(stopwords.words("french")))
french_stopwords[0:10]


def generate_bigrams_nltk(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [token for token in tokens if token.isalnum() and token not in french_stopwords]
    bigrams = list(ngrams(filtered_tokens, 2))
    return bigrams

#generate_bigrams_nltk(df["texte_net"].iloc[0])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilien/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [110]:
# Count bigrams:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=french_stopwords, ngram_range=(2, 2), max_features=300)
bigrammes = (
    pd.DataFrame(
        vectorizer.fit_transform(df["texte_net"]).toarray(),
        columns=vectorizer.get_feature_names_out(),
    )
    .T.sum(axis=1)
    .sort_values(ascending=False)
)
bigrammes

bancs groupe              15134
président parole           9976
applaudissements bancs     9119
premier ministre           8117
bancs groupes              6324
                          ...  
fin année                   282
monsieur secrétaire         281
danièle obono               278
grand débat                 277
commission européenne       276
Length: 300, dtype: int64

## Représenter les textes

### Présence de mots

In [119]:
df["dim1"] = df["texte_net"].str.contains("intelligence artificielle")
df["dim2"] = df["texte_net"].str.contains("science")
df[["dim1","dim2"]].replace({True:1,False:0}).head()

,dim1,dim2
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


### Vecteur brut : Document term matrix / tableau

In [130]:
from sklearn.feature_extraction.text import CountVectorizer

# créer mon object de ML
vectorizer = CountVectorizer(stop_words=french_stopwords, 
                             ngram_range=(1, 1), 
                             max_features=800)

# appliquer sur les données
X = vectorizer.fit_transform(df["texte_net"])
X = pd.DataFrame(X.toarray(),columns=list(vectorizer.get_feature_names_out()))

In [136]:
# X.iloc[2]

### Une version un peu plus avancée

- Term Frequency-Inverse Document Frequency
    - Amélioration du DTM
- Approche souvent utilisée pour mettre en valeur les mots les plus spécifiques
- `Scikit-learn` a `TfidfVectorizer`

$$\text{TF-IDF}(t, d, D) = \left( \frac{f_{t,d}}{n_d} \right) \times \log \left(\frac{N}{\text{df}_t} \right)
$$

In [142]:
from sklearn.feature_extraction.text import TfidfVectorizer

# créer un objet
vectorizer = TfidfVectorizer(stop_words=french_stopwords, 
                             ngram_range=(1, 1), 
                             max_features=800)

# applique 
X = vectorizer.fit_transform(df["texte_net"])

# mettre en forme
X = pd.DataFrame(X.toarray(),columns=list(vectorizer.get_feature_names_out()))
X.loc[100].sort_values()

000           0.000000
pauvreté      0.000000
pays          0.000000
pense         0.000000
permet        0.000000
                ...   
protection    0.130034
relance       0.155877
commission    0.213689
écologique    0.323248
transition    0.525928
Name: 100, Length: 800, dtype: float64

In [ ]:
len(vectorizer.get_feature_names_out())

Faire la matrice TF-IDF, identifier les mots qui ont le score le plus important

## Distance entre deux textes

In [149]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

X = vectorizer.fit_transform(df["texte_net"])
cosine_similarity(X[0], X[4])

array([[0.20955574]])

In [ ]:
distances = pd.DataFrame(pairwise_distances(X, metric="cosine"))

In [ ]:
distances[10].idxmax()

## Application : Faire un nuage de mots avec WordCloud

Un coup d'oeil à la [documentation](https://amueller.github.io/word_cloud/)